In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import requests
import time
import json
import keras
import warnings
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

# Display preference
warnings.filterwarnings('ignore')

In [15]:
def scroll(driver, option, timeout):
    
    if option == 1:
    
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
    
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait
            time.sleep(timeout)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
        
            if new_height == last_height:
            
                break
        
            last_height = new_height
    else:
        
        # Used to scroll webpage and scrape occupation names
        for i in range(9):
            
            # Get element
            load_more_button = driver.find_element_by_xpath('/html/body/main/div/div[1]/article/div[5]/div/div/section/div/button/div')
            driver.execute_script("arguments[0].click();", load_more_button)
            
            # Wait
            time.sleep(timeout)        

In [16]:
# Set options
options = Options()
options.set_preference('permissions.default.image', 2)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)

def all_links(url, scroll_option=0, action=0):
    
    # Setup driver
    driver = webdriver.Firefox(options=options, executable_path='D:/Users/bengriffith/Desktop/geckodriver.exe')

    # Wait before throwing an exception
    driver.implicitly_wait(20)

    # Open the page
    driver.get(url)
    
    # Start scrolling
    scroll(driver, scroll_option, 10)
    
    # Beautiful Soup parses page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Close the driver
    driver.close()

    if action == 1:
    
        # Empty dictionary to store m from page source
        get_m = {}
    
        # Counter for get_m dictionary
        get_m_counter = 0

        # Looping through all the a elements in the page source
        for link in soup.find_all('a', {'class': 'iusc'}):
    
            # Add values to dictionary
            get_m[get_m_counter] = link.get('m')
        
            # Increment counter
            get_m_counter += 1
        
        # Convert from string to dictionary
        for key, value in get_m.items():
            get_m[key] = json.loads(value)
        
        # Empty list to store image links
        links = []
    
        # Loop through get_m dictionary and add image links to links list
        for value in get_m.values():
            links.append(value['murl'])
        
        return links
    else:
        return soup

In [17]:
# List of search phrases
search_phrases = []

In [18]:
# Popular names url
name_url = 'https://www.ssa.gov/oact/babynames/decades/century.html'

# Beautiful Soup object
soup_name = all_links(name_url, 1)

# Get td tag
td_tag = soup_name.select('.t-stripe > tbody > tr > td')

# Add name to search phrases
search_phrases += [name.get_text().lower() for name in td_tag if name.get_text().isalpha()]

In [19]:
# Best ranked jobs url
occupation_url = 'https://money.usnews.com/careers/best-jobs/rankings/the-100-best-jobs'

# Beautiful Soup object
soup_occupation = all_links(occupation_url)
    
# Get a tag
a_tag = soup_occupation.select('.card-body-container > div > h2 > a')

# Add occupation to search phrases
search_phrases += [occupation.get_text().lower() for occupation in a_tag]

In [20]:
# List for image links
image_links = []

# Get image links for each search phrase
for phrase in search_phrases:
    
    # Update url
    url = 'https://www.bing.com/images/search?q=' + phrase + '&qs=n&form=QBIR&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7%20filterui%3Aface-face&sp=-1&pq=' + phrase + '&sc=8-9&sk=&cvid=8C2D8EC9A35A4948980A61669549C180'
    
    # Add image links
    image_links += all_links(url, 1, 1)

KeyboardInterrupt: 

In [20]:
len(image_links)

18825

In [21]:
for image_link in image_links[:10]:
    print(image_link)

https://upload.wikimedia.org/wikipedia/commons/a/ad/Angelina_Jolie_2_June_2014_%28cropped%29.jpg
https://2.bp.blogspot.com/_M9roea07aEo/TRnTECqVsnI/AAAAAAAAAes/Kih6d-dRBvk/s1600/jennifer-lopez.jpg
http://1.bp.blogspot.com/-ecLBfr5juf0/TZEvBL43nfI/AAAAAAAABWk/BqzSzkfrpzQ/s640/britney_spears_new_style_face.jpg
http://3.bp.blogspot.com/_IA5nokOFh84/TMKV5YVhRhI/AAAAAAAAEe8/QMLzDZ6LRqU/s1600/lindsay-lohan.jpg
http://1.bp.blogspot.com/-0Jn4T9wSUx8/TdHrTyyo23I/AAAAAAAAFxs/68tE6_Y7-rY/s1600/bailee-madison-2nd-annual-fashion-with-a-passion-teen-celebrity-fashion-show-to-benefit-camp-ronald-mcdonald-arrivals-qPJr2Z.jpg
https://lh5.googleusercontent.com/proxy/KT-yu9mAlCmdmmPPstVB-EtQF8K_8l7weO-XedDcHrt6E2UHmuaQOUqEONaV-F721H5Et9uMLfK2S0yM-_0BS8dcUNvfKELvupuvCKlogPc0LcmJVoygfnhvNfwUxWWlvBZ4saH6tgQ=s0-d
http://2.bp.blogspot.com/-Q9qEniziUIQ/TpZWFGLdj-I/AAAAAAAABCA/ghOH9LFiPZc/s640/IMG_2150.JPG
http://1.bp.blogspot.com/-0Jn4T9wSUx8/TdHrTyyo23I/AAAAAAAAFxs/68tE6_Y7-rY/s1600/bailee-madison-2nd-annual-

In [22]:
# Header to accompany get request
headers = {'user-agent': 'scraper - school project (bengriffith@outlook.com)'}

# Counter to be used in image file name
image_counter = 0

# Write images to disk
for image_link in image_links[:3]:
    with open('images/face_' + str(image_counter) + '.png', 'wb') as f:
        response = requests.get(image_link, headers=headers)
        f.write(response.content)
    image_counter += 1

In [23]:
image_data = []

for filename in os.listdir('images/')[1:2]:
    image = cv2.imread('images/' + filename)
    image = cv2.resize(image, (250, 250)).flatten()
    image_data.append(image)

In [58]:
image_data = np.array(image_data, dtype='float') / 255.0

In [24]:
prototxt = 'supporting/deploy.prototxt'
model = 'supporting/res10_300x300_ssd_iter_140000.caffemodel'

In [37]:
net = cv2.dnn.readNetFromCaffe(prototxt, model)

image = cv2.imread('images/face_1.png')
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

In [38]:
net.setInput(blob)
detections = net.forward()

In [39]:
for i in range(detections.shape[2]):
    
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.5:
        
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')
        
        text = '{:.2f}%'.format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
        
cv2.imshow('Output', image)
cv2.waitKey(0)

-1